<h1>3. Skill Prediction en LinkedIN

**3.0 Importacines necesarias**

Se importan librerías y métodos necesarios para el correcto funcionamiento de las activades a desarrollar dentro de ésta sección.

In [62]:
#!usr/bin/env python
# -*- coding: utf-8 -*-
from scipy.sparse import csr_matrix
import re
import numpy as np
from sklearn.utils import shuffle
from sklearn.utils import column_or_1d
from clasificadores import *

**3.1 Construcción de dataset**

Primeramente, se crea la función *generar_Z*, la cual, a partir de los archivos *user_skill* y *skill_id*, construye una matriz dispersa con los datos provistos en dicho archivo. Tal matriz será denotada de aquí en adelante como Z.

In [2]:
#Se define funcion para construir matriz Z
def generar_Z(arch1, arch2):
    #Variables que contienen la cantidad de perfiles y de competencias, respectivamente
    n_perfiles = 0
    n_competencias = 0
    
    #Se determina la cantidad de perfiles
    with open(arch1) as perfiles:
        n_perfiles = sum(1 for _ in perfiles)
    
    #Se determina la cantidad de competencias
    with open(arch2) as competencias:
        n_competencias = sum(1 for _ in competencias)
        
    #Se crea fisicamente la matriz Z
    Z = np.zeros(shape=(n_perfiles, n_competencias), dtype=np.int)
    
    #Se procede a 'trasladar' los valores de los archivos a Z
    with open(arch1) as perfiles:
        for line in perfiles:
            datos = re.split(':|,|\n', line)
            n_perfil = int(datos[0])
            for n_competencia in datos[1:-1]:
                Z[n_perfil, int(n_competencia)] = 1
    
    Z = csr_matrix(Z)
    return Z

Luego, se procede a construir la matriz Z con que se trabajará de ahora en adelante.

In [3]:
Z = generar_Z('user_skill', 'skill_id')

A partir del resultado anterior, se puede obtener la cantidad de perfiles y de competencias que integran los dataset.

In [4]:
print 'Se cuenta con un dataset compuesto por', Z.shape[0], 'perfiles y', Z.shape[1], 'competencias.'

Se cuenta con un dataset compuesto por 7890 perfiles y 14544 competencias.


**3.2 Construcción de set de entrenamiento y set de pruebas**

A partir de la matriz Z generada en la sección anterior, se procede a construir los conjuntos de entrenamiento y de prueba. Para ello, se seleccionan registros desde Z en forma azarosa de tal forma que el conjunto de entrenamiento estará compuesto por el 60% de los registros del dataset original, mientras que el conjunto de prueba estará integrando por el 40% de los registros de dicho dataset. Considerando que Z acumula 7890 perfiles, el conjunto de entrenamiento tendrá 4734 registros y el de prueba, 3156 registros.

Para realizar la tarea descrita, se sigue el procedimiento presentado a continuación:

In [5]:
#Se permutan aleatoriamente las filas de la matriz Z
Z = shuffle(Z, random_state=0)

#Se construyen conjunto de entrenamiento y de prueba, respectivamente
Z_tr = Z[0:int(0.6*Z.shape[0])]
Z_ts = Z[int(0.6*Z.shape[0]):]

**3.3 Cantidad de perfiles por competencia**

En ésta sección, se desea construir un gráfico en el que sea posible apreciar la cantidad de perfiles que declaran una determinada competencia. Para esto, se lleva a cabo el siguiente procedimiento:

In [6]:
#NOTA: La siguiente pieza de codigo demora alrededor de 20 segundos en ejecutarse

#Se obtiene el nombre, identificador y cantidad de perfiles asociado a cada competencia, y se 
#almacenan en una lista de tuplas

lista_competencias = []
with open('skill_id') as competencias:
    for line in competencias:
        dato = re.split(' :|:|\n', line)
        dato = dato[0:-1]
        
        if (len(dato) > 2):
            nombre_competencia = ''
            for token in dato[0:-1]:
                nombre_competencia += token
            id_competencia = int(dato[-1])
            cantidad_perfiles = Z[:, id_competencia].sum()
            lista_competencias.append((nombre_competencia, id_competencia, cantidad_perfiles))
            
        else:
            if (dato[0] != ''):
                nombre_competencia = dato[0]
                id_competencia = int(dato[1])
                cantidad_perfiles = Z[:, id_competencia].sum()
                lista_competencias.append((nombre_competencia, id_competencia, cantidad_perfiles))
            
#Finalizado el proceso anterior, se ordena la lista competencias por su frecuencia
lista_competencias.sort(key=lambda tup: tup[2], reverse=True)

#A continuacion, se muestra un ranking con las 10 competencias mas frecuentes
print('A continuación, se muestra un ranking con las 10 competencias más frecuentes (con su id entre \nparéntesis), acompañadas por la cantidad de perfiles en que es declarada. Es importante considerar que sólo se han tomado en cuenta aquellas competencias cuyo nombre es no nulo.\n')
contador = 0
for comp in lista_competencias:
    print contador+1,'.', comp[0], '(', comp[1],')',':', comp[2]
    contador += 1
    if (contador > 9):
        break

A continuación, se muestra un ranking con las 10 competencias más frecuentes (con su id entre 
paréntesis), acompañadas por la cantidad de perfiles en que es declarada. Es importante considerar que sólo se han tomado en cuenta aquellas competencias cuyo nombre es no nulo.

1 . Microsoft Office ( 29 ) : 3081
2 . Liderazgo de equipos ( 32 ) : 2761
3 . Gestión de proyectos ( 43 ) : 2204
4 . Microsoft Excel ( 71 ) : 2191
5 . Team Leadership ( 1 ) : 2109
6 . Estrategia empresarial ( 47 ) : 1868
7 . SQL ( 55 ) : 1782
8 . Planificación estratégica ( 45 ) : 1717
9 . Inteligencia empresarial ( 49 ) : 1673
10 . Inglés ( 33 ) : 1647


**3.4 Predicción de la competencia *Microsoft Office***

**3.4.1 Ajuste de conjuntos de entrenamiento y prueba**

Se desea predecir la existencia de la competencia *Microsoft Office* en un perfil determinado, en base a las competencias restantes que el usuario declaró. Se ha seleccionado dicha competencia debido a su alta frecuencia de aparición en los perfiles. Para el posterior entrenamiento de clasificadores, se ajustan los sets de entrenamiento y de prueba construídos previamente para así poder abordar este problema. Notar que para evaluar el rendimiento de cada clasificador, se usarán las funciones previamente creadas para abordar el problema 2. Además, se usa la siguiente notación:

clase 1: Implica que la competencia *Microsoft Office* efectivamente es declarada un un perfil.  
clase 0: Caso contrario.

Se procede de la siguiente manera:

In [52]:
#Para el set de entrenamiento, se construyen las matrices X_tr e y_tr a partir de Z_tr
#Recordar que Microsoft Office corresponde a la competencia numero 29
y_tr = column_or_1d(Z_tr[:, 29].toarray())
X_tr = csr_matrix(np.delete(Z_tr.toarray(), 29, 1))

#Se procede de forma analoga con el conjunto de entrenamiento, construyendo X_ts e y_ts
#a partir de Z_ts
y_ts = column_or_1d(Z_ts[:, 29].toarray())
X_ts = csr_matrix(np.delete(Z_ts.toarray(), 29, 1))

**3.4.2 Entrenamiento/ajuste de clasificador Bayesiano Ingenuo Binario**

En base a los conjuntos de datos elaborados en la sección anterior, se procede a entrenar/ajustar un clasificador Bayesiano Binario. Posterior a ello, se aplica el modelo generado sobre los datos de entrenamiento y de prueba con el fin de determinar su precisión sobre estos conjuntos.

In [63]:
NAIVE_BAYES(X_tr, y_tr, X_ts, y_ts)

Precisión datos de entrenamiento BernoulliNB: 0.849599
Precisión datos de prueba BernoulliNB: 0.794360
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.81      0.87      0.84      1914
    clase 0       0.77      0.68      0.72      1242

avg / total       0.79      0.79      0.79      3156



BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

**3.4.3 Entrenamiento/ajuste de clasificador Bayesiano Ingenuo Multinomial**

Se ejecuta el mismo procedimiento de la sección previa, pero considerando un clasificador Bayesiano Ingenuo Multinomial.

In [64]:
MULTINOMIAL(X_tr, y_tr, X_ts, y_ts)

Precisión datos de entrenamiento MULTINOMIAL: 0.847909
Precisión datos de prueba MULTINOMIAL: 0.786122
Análisis detallado de resultados sobre set de prueba:
             precision    recall  f1-score   support

    clase 1       0.83      0.81      0.82      1914
    clase 0       0.72      0.75      0.73      1242

avg / total       0.79      0.79      0.79      3156



MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)